https://optuna.readthedocs.io/en/stable/tutorial/10_key_features/005_visualization.html#sphx-glr-download-tutorial-10-key-features-005-visualization-py

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import torch
import pandas as pd
from pathlib import Path
import optuna
from reprpo.hp.helpers import optuna_df

In [3]:
from reprpo.training import train
from reprpo.experiments import experiment_configs
from reprpo.hp.space import search_spaces

[2024-10-03 12:43:18,833] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


## Objective

In [4]:
SEED=42
key_metric = "acc_gain_vs_ref/oos"
torch.manual_seed(SEED)
DEVICE = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [5]:
# silence please
import os
from loguru import logger
logger.remove()
logger.remove()
logger.add(os.sys.stderr, level="WARNING")

os.environ["WANDB_MODE"] = "disabled"
os.environ["HF_DATASETS_OFFLINE"] = "1"
os.environ["TQDM_DISABLE"] = "true"

In [6]:
f_db = f"sqlite:///optuna.db"
f = f_db.replace('sqlite:///', './')
print(f)
Path(f).parent.mkdir(parents=True, exist_ok=True)
f_db

./optuna.db


'sqlite:///optuna.db'

In [7]:
# print(f'to visualise run in cli\ncd nbs\noptuna-dashboard {f_db}')

In [8]:
from reprpo.hp.target import override, default_tuner_kwargs
from reprpo.experiments import experiment_configs
import copy
import wandb

import optuna.pruners
from optuna_integration.wandb import WeightsAndBiasesCallback

In [9]:


import functools

def list2tuples(d):
    for k, v in d.items():
        if isinstance(v, list):
            d[k] = tuple(v)
    return d

def objective_func(kwargs, trial, starter_experiment_name):
    cfg = copy.deepcopy(experiment_configs[starter_experiment_name][1])
    override(cfg, default_tuner_kwargs)
    
    override(cfg, kwargs)
    kwargs = list2tuples(kwargs)
    r = train(cfg, trial=trial)
    return r

def objective(trial: optuna.Trial, starter_experiment_name, trial2args, key_metric=key_metric) -> float:
    kwargs = trial2args(trial)
    r = objective_func(kwargs, trial, starter_experiment_name)
    return r[key_metric]



## Opt

Note on pruning. It's only really usefull with validation metrics and for long jobs over many epochs. I've got a small proxy job so there is no need.

In [10]:
MAX_TRIALS= 250


for starter_experiment_name, trial2args in search_spaces.items():

    study_name = f"{starter_experiment_name}"
    study = optuna.create_study(
        study_name=study_name,
        direction="maximize",
        load_if_exists=True,
        storage=f_db,
        sampler=optuna.samplers.TPESampler(seed=SEED),
        # pruner=optuna.pruners.NopPruner(),
    )

    n = 0
    if len(study.trials)>0:
        df = study.trials_dataframe().query('state == "COMPLETE"').sort_values('value', ascending=False)
        n = len(df)
        print(f"loaded {n} {study_name} trials")
        # print('study.best_trial', study.best_trial)

        df_res = optuna_df(study, key_metric)
        print(df_res.to_markdown())

    
    if n < MAX_TRIALS:
        _objective = functools.partial(objective, key_metric=key_metric, starter_experiment_name=starter_experiment_name, trial2args=trial2args)

        study.optimize(_objective, 
                    n_trials=MAX_TRIALS, 
                    gc_after_trial=True, 
                    catch=(AssertionError, OSError, RuntimeError, KeyError)
        )

    print('='*80)

[I 2024-10-03 12:44:21,375] A new study created in RDB with name: projgrad2
[W 2024-10-03 12:44:22,090] Trial 0 failed with parameters: {'β': 0.3745401188473625, 'reverse_pref': True, 'scale_orth': True, 'weight_dim': 0, 'neg_slope': 0.01, 'mag_clip': None, 'lr': 1.9762189340280066e-05, 'collect_input': False, 'collect_hs': False} because of the following error: KeyError('projgrad2').
Traceback (most recent call last):
  File "/workspace/repr-preference-optimization/.venv/lib/python3.11/site-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "/tmp/ipykernel_1099294/2880211868.py", line 20, in objective
    r = objective_func(kwargs, trial, starter_experiment_name)
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_1099294/2880211868.py", line 10, in objective_func
    cfg = copy.deepcopy(experiment_configs[starter_experiment_name][1])
                        ~~~~~~~~~

[I 2024-10-03 12:48:13,538] A new study created in RDB with name: side-ether-prefvec
wandb: WARNING Disabling the wandb service is deprecated as of version 0.18.0 and will be removed in version 0.19.0.
[I 2024-10-03 12:51:31,978] Trial 0 finished with value: 1.0076923076923077 and parameters: {'lr': 1.329291894316217e-05, 'collect_input': True, 'collect_hs': True, 'nb': 5, 'Htype': 'etherplus', 'flip_side': False, 'reduction': 167, 'β': 2.177484667394932e-05, 'use_orth_loss': False, 'use_angle_loss': False, 'use_dpo_loss': True, 'use_nll_loss': True, 'weight_tokens': False}. Best is trial 0 with value: 1.0076923076923077.



| acc_inc/eval_ds [pp]                                                                                                                                                                                                            |   train |   test |   oos |   rnd |
|:--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|--------:|-------:|------:|------:|
| ReprPO collect_hs=True collect_input=True loss.use_angle_loss=False loss.use_dpo_loss=True loss.use_nll_loss=True loss.use_orth_loss=False loss.β=0.00 lr=0.00 transform.Htype=etherplus transform.nb=5 transform.reduction=167 |   1.653 |      0 | 0.769 | 5.263 |


[I 2024-10-03 12:54:37,159] Trial 1 finished with value: 0.6557692307692308 and parameters: {'lr': 2.334586407601622e-05, 'collect_input': True, 'collect_hs': False, 'nb': 2, 'Htype': 'etherplusHH', 'flip_side': True, 'reduction': 61, 'β': 4.125104169651939e-06, 'use_orth_loss': True, 'use_angle_loss': False, 'use_dpo_loss': False, 'use_nll_loss': False, 'weight_tokens': False}. Best is trial 0 with value: 1.0076923076923077.



| acc_inc/eval_ds [pp]                                                                                                                   |   train |    test |     oos |    rnd |
|:---------------------------------------------------------------------------------------------------------------------------------------|--------:|--------:|--------:|-------:|
| ReprPO collect_input=True loss.use_angle_loss=False loss.β=0.00 lr=0.00 transform.flip_side=True transform.nb=2 transform.reduction=61 | -26.446 | -25.984 | -34.423 | 17.544 |


...


| acc_inc/eval_ds [pp]                                                                                                                    |   train |   test |    oos |   rnd |
|:----------------------------------------------------------------------------------------------------------------------------------------|--------:|-------:|-------:|------:|
| ReprPO loss.use_orth_loss=False loss.β=0.52 lr=0.00 transform.Htype=oft transform.flip_side=True transform.nb=31 transform.reduction=19 |   3.306 |      0 | 13.654 | 5.263 |


[I 2024-10-04 00:00:33,448] Trial 205 finished with value: 1.1134615384615385 and parameters: {'lr': 0.0004082554597602717, 'collect_input': False, 'collect_hs': False, 'nb': 31, 'Htype': 'oft', 'flip_side': True, 'reduction': 25, 'β': 0.2210893250648101, 'use_orth_loss': False, 'use_angle_loss': True, 'use_dpo_loss': False, 'use_nll_loss': False, 'weight_tokens': False}. Best is trial 192 with value: 1.169230769230769.



| acc_inc/eval_ds [pp]                                                                                                                    |   train |   test |    oos |    rnd |
|:----------------------------------------------------------------------------------------------------------------------------------------|--------:|-------:|-------:|-------:|
| ReprPO loss.use_orth_loss=False loss.β=0.22 lr=0.00 transform.Htype=oft transform.flip_side=True transform.nb=31 transform.reduction=25 |   3.306 |      0 | 11.346 | -5.263 |


[I 2024-10-04 00:03:47,063] Trial 206 finished with value: 1.169230769230769 and parameters: {'lr': 0.0006032647595682293, 'collect_input': False, 'collect_hs': False, 'nb': 32, 'Htype': 'oft', 'flip_side': True, 'reduction': 31, 'β': 0.30490860430415506, 'use_orth_loss': False, 'use_angle_loss': True, 'use_dpo_loss': False, 'use_nll_loss': False, 'weight_tokens': False}. Best is trial 192 with value: 1.169230769230769.



| acc_inc/eval_ds [pp]                                                                                                                    |   train |   test |    oos |    rnd |
|:----------------------------------------------------------------------------------------------------------------------------------------|--------:|-------:|-------:|-------:|
| ReprPO loss.use_orth_loss=False loss.β=0.30 lr=0.00 transform.Htype=oft transform.flip_side=True transform.nb=32 transform.reduction=31 |   2.479 | -1.575 | 16.923 | 12.281 |


In [ ]:
print(wandb.run.get_url())

## plot

In [ ]:
# You can use Matplotlib instead of Plotly for visualization by simply replacing `optuna.visualization` with
# `optuna.visualization.matplotlib` in the following examples.
from optuna.visualization.matplotlib import plot_contour
from optuna.visualization.matplotlib import plot_edf
from optuna.visualization.matplotlib import plot_intermediate_values
from optuna.visualization.matplotlib import plot_optimization_history
from optuna.visualization.matplotlib import plot_parallel_coordinate
from optuna.visualization.matplotlib import plot_param_importances
from optuna.visualization.matplotlib import plot_rank
from optuna.visualization.matplotlib import plot_slice
from optuna.visualization.matplotlib import plot_timeline

In [ ]:
search_spaces.keys()

In [ ]:
starter_experiment_name = 'projgrad'
trial2args = search_spaces[starter_experiment_name]

study_name = f"{starter_experiment_name}"
study = optuna.create_study(
    study_name=study_name,
    direction="maximize",
    load_if_exists=True,
    storage=f_db,
    sampler=optuna.samplers.TPESampler(seed=SEED),
    pruner=optuna.pruners.NopPruner(),
)
print('study.best_trial', study.best_trial)
df = study.trials_dataframe().query('state == "COMPLETE"').sort_values('value', ascending=False)
print(len(df))
plot_optimization_history(study)

In [13]:
# plot_timeline(study)

In [29]:
# plot_intermediate_values(study)

In [30]:
# plot_contour(study)


In [ ]:
plot_slice(study)


In [ ]:
plot_param_importances(study)

In [ ]:
plot_intermediate_values(study)

In [ ]:
plot_parallel_coordinate(study)

### Apendix 1: dataclass 2 optuna

In [20]:
# import inspect
# import typing
# from typing import Literal

# def optuna_suggest_from_dataclass(t):
#     n = t.__name__
#     print(f'## {n}')
#     sig = inspect.signature(t)
#     for name, param in sig.parameters.items():
#         if param.annotation== bool:
#             print(f'"{name}": trial.suggest_categorical("{name}", [True, False]),')
#         elif param.annotation==int:
#             print(f'"{name}": trial.suggest_int("{name}", 1, 10),')
#         elif param.annotation ==float:
#             print(f'"{name}": trial.suggest_float("{name}", 0.1, 10.0),')
#         elif param.annotation == str:
#             print(f'"{name}": trial.suggest_categorical("{name}", ["a", "b", "c"]),')
#         elif param.annotation == tuple:
#             print(f'"{name}": trial.suggest_categorical("{name}", [(1, 2), (3, 4), (5, 6)]),')
#         elif typing.get_origin(param.annotation) == Literal:
#             print(f'"{name}": trial.suggest_categorical("{name}", {param.annotation.__args__}),')
#         else:
#             print(f"!!Unknown type {param}")
#             # print(name, param.default, param.annotation)

# optuna_suggest_from_dataclass(ReprPOConfig)
# for t in Transforms:
#     print(f'## {t}')
#     optuna_suggest_from_dataclass(t.value)
# for l in Losses:
#     print(f'## {l}')
#     optuna_suggest_from_dataclass(l.value)


# optuna_suggest_from_dataclass(DPOProjGradConfig)